In [1]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import lxml
import html5lib

In [2]:
import os

In [3]:
path = "../Data/games/"
game_files = os.listdir(path)

In [4]:
game_files

['apr-jun-2023.html',
 'oct-des-2023.html',
 'jan-mar-2023.html',
 'jul-sep-2023.html',
 'jul-sep-2022.html',
 'jan-mar-2022.html',
 'oct-des-2022.html',
 'apr-jun-2022.html']

In [5]:
tables = []
def parse_file(fn):
    with open(f"../Data/games/{fn}", 'r') as f:
        soup = bs(f, 'html.parser')
    # table = soup.find('table',{'class':"wikitable"})
    df=pd.read_html(str(soup), flavor='html5lib')
    return pd.DataFrame(df[0])
for i in game_files:
    df = parse_file(i)
    tables.append(df)

In [6]:
tables

[         Month  Day                                 Title   
 0    A P R I L    4              GrimGrimoire OnceMore[a]  \
 1    A P R I L    4                       Meet Your Maker   
 2    A P R I L    4                       Road 96: Mile 0   
 3    A P R I L    6                 Curse of the Sea Rats   
 4    A P R I L    6                    Dungeons of Aether   
 ..         ...  ...                                   ...   
 148    J U N E   30                         Goodbye World   
 149    J U N E   30  Master Detective Archives: Rain Code   
 150    J U N E  TBA                                AFL 23   
 151    J U N E  TBA                            Cricket 24   
 152    J U N E  NaN                                   NaN   
 
                      Platform(s)             Genre(s)   
 0                   NS, PS4, PS5   Real-time strategy  \
 1        Win, PS4, PS5, XBO, XSX             Strategy   
 2    Win, NS, PS4, PS5, XBO, XSX            Adventure   
 3    Win, NS, PS4, PS

In [16]:
df = pd.concat(tables).rename(columns={"Developer(s)": "Developers", "Publisher(s)": "Publishers"})

In [17]:
df = df[['Title', 'Developers', 'Publishers']]

In [18]:
df[["Title", "Developers", "Publishers"]] = df[["Title", "Developers", "Publishers"]].apply(lambda x: [
    str(x["Title"]),
    str(x['Developers']),
    str(x['Publishers'])
    ], axis=1, result_type='expand')

In [19]:
def Split(s, *args):
    return [str(i).split(s) for i in args]
df[["Developers", "Publishers"]] = df[["Developers", "Publishers"]].apply(lambda x: Split(", ", *x), axis=1, result_type='expand')

In [20]:
df

,Title,Developers,Publishers
0,GrimGrimoire OnceMore[a],[Vanillaware],[NIS America]
1,Meet Your Maker,[Behaviour Interactive],[Behaviour Interactive]
2,Road 96: Mile 0,[DigixArt],[Ravenscourt]
3,Curse of the Sea Rats,[Petoons Studio],[PQube]
4,Dungeons of Aether,"[Dan Fornace, AmpersandBear]",[Dan Fornace]
...,...,...,...
273,Of Bird and Cage,[Capricia Productions],[All in! Games]
274,Outriders Worldslayer,[People Can Fly],[Square Enix]
275,Psikyo Shooting Stars Alpha[c],[Zerodiv],[City Connection]
276,Rabbids: Party of Legends[a],[Ubisoft Chengdu],[Ubisoft]


In [21]:
df = df.explode("Developers").explode('Publishers')

In [22]:
df

,Title,Developers,Publishers
0,GrimGrimoire OnceMore[a],Vanillaware,NIS America
1,Meet Your Maker,Behaviour Interactive,Behaviour Interactive
2,Road 96: Mile 0,DigixArt,Ravenscourt
3,Curse of the Sea Rats,Petoons Studio,PQube
4,Dungeons of Aether,Dan Fornace,Dan Fornace
...,...,...,...
273,Of Bird and Cage,Capricia Productions,All in! Games
274,Outriders Worldslayer,People Can Fly,Square Enix
275,Psikyo Shooting Stars Alpha[c],Zerodiv,City Connection
276,Rabbids: Party of Legends[a],Ubisoft Chengdu,Ubisoft


In [25]:
import numpy as np
import json

In [27]:
with open('games.json', 'w') as f:
    json.dump(list(np.unique(df['Title'])), f)

In [28]:
with open('Developers.json', 'w') as f:
    json.dump(list(np.unique(df['Developers'])), f)

In [29]:
with open('Publishers.json', 'w') as f:
    json.dump(list(np.unique(df['Publishers'])), f)